# Aprendizado Profundo - UFMG

## Preâmbulo

O código abaixo consiste dos imports comuns. Além do mais, configuramos as imagens para ficar de um tamanho aceitável e criamos algumas funções auxiliares. No geral, você pode ignorar a próxima célula.

In [1]:
# !pip install mxnet-cu100==1.4.1

In [2]:
# -*- coding: utf8

import matplotlib.pyplot as plt

import mxnet as mx
import mxnet.ndarray as nd

import numpy as np

plt.rcParams['figure.figsize']  = (18, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4

In [3]:
plt.ion()

plt.style.use('seaborn-colorblind')
plt.rcParams['figure.figsize']  = (12, 8)

O código de GD abaixo está pré-preparado para a primeira parte da tarefa. Não precisa mudar o mesmo!

In [4]:
def gd(d_fun, loss_fun, X, Y, lambda_=10, tol=0.00001, max_iter=100):
    '''
    Executa Gradiente Descendente. Aqui:
    
    Parâmetros
    ----------
    d_fun : é uma função de derivadas
    loss_fun : é uma função de perda
    X : é um vetor de fatores explanatórios.
        Copie seu código de intercepto da primeira aula.
        para adicionar o intercepto em X.
    y : é a resposta
    lambda : é a taxa de aprendizad
    tol : é a tolerância, define quando o algoritmo vai parar.
    max_ter : é a segunda forma de parada, mesmo sem convergir
              paramos depois de max_iter iterações.
    '''
    Theta = nd.ones(shape=(Y.shape[1], X.shape[1]))
    
    old_lf = np.inf
    print('Iter {}; Loss = '.format(0), old_lf)
    
    i = 0
    while True:
        # Computar as derivadas
        Grad = d_fun(X, Theta, Y)
        
        # Atualizar
        Theta_novo = Theta - lambda_ * Grad
        
        # Parar quando o erro convergir
        lf = loss_fun(X, Theta, Y)
        print(lf, old_lf)
        if nd.abs(old_lf - lf) <= tol:
            break
        
        # Atualizar parâmetros e erro
        Theta = Theta_novo
        old_lf = lf
        
        # Informação de debug
        i += 1
        print('Iter {}; Loss = '.format(i), lf[0])
        if i == max_iter:
            break
    return Theta

Para testar o resultado dos seus algoritmos vamos usar o módulo testing do numpy.

In [5]:
from numpy.testing import assert_equal
from numpy.testing import assert_almost_equal
from numpy.testing import assert_array_almost_equal

## Aula 03 - Softmax para Imagens

Continuando da aula anterior, vamos implementar uma regressão logística (softmax) para várias classes. Além do mais, vamos fazer uso da mesma para algumas tarefas classificação de imagens.

Para iniciar a transição para o mundo de Deep Learning, vamos implementar uma nova logística (não será mais do zero) usando as camadas que o mxnet já traz pronta.

Antes de iniciar o notebook, sugiro uma revisão do [Capítulo 3](d2l.ai).

## Softmax em MXNET/Gluon

A função softmax pode ser utilizada para problemas multiclasse. No exemplo abaixo, temos a função softmax exemplificada. Diferente dos casos anteriores, aqui nós temos uma matriz de parâmetros: $\mathbf{\Theta}$. Cada coluna da matriz $\mathbf{\Theta}_y$ contém os parâmetros para uma classe $y$.

$$p(y|\mathbf{x}_i, \mathbf{\theta}_y) = \mathrm{softmax}(\mathbf{x}^t_i) = \frac{\exp(\mathbf{x}^t_i \mathbf{\theta}_y)}{\sum_{y'} \exp(\mathbf{x}^t_{i} \mathbf{\theta}_{y'})}$$

Vamos pensar no caso que temos 10 classes e 20 atributos:
  * O tamanho e $\mathbf{X}$ é `(n, 20)`, `n` é o número de exemplos
  * O tamanho e $\mathbf{\Theta}$ é `(c, 20)`, onde `c` é o número de classes
  * $\mathbf{\Theta}^t$ é `(20, c)`
  * $\mathbf{X} \mathbf{\Theta}^t$ é `(n, c)`. Uma probabilidade para cada classe. Tal produto interno é representado por uma linha de $\mathbf{X}$, $\mathbf{x}^t_i$ multiplicado pela coluna de $\mathbf{\Theta}$, $\mathbf{\theta}_y$.
  
O softmax é basicamente uma matriz onde aplicamos uma exponencial em toda célula: $e^{\mathbf{X} \mathbf{\Theta}^t}$. Depois normalizamos por classe. Assim voltamos para probabilidades.

Primeiro, sem usar o gluon ainda, implemente:
   1. Uma função softmax
   1. Uma função de perda
   1. Derivadas em mxnet

Antes disso, vamos usar nossos blobs de sempre. Essa deve ser a última aula com os mesmo :-(

In [6]:
from sklearn import datasets
state = np.random.seed(20190187)

X, y = datasets.make_blobs(n_samples=200, n_features=20, centers=2)

No nosso novo exemplo temos 200 amostras, 20 features e 10 classes!

In [7]:
X.shape

(200, 20)

In [8]:
len(set(y))

2

Implemente a função softmax.

In [9]:
def softmax(X, Theta):
    '''
    Aqui Theta é a matriz de parâmetros e X uma matriz.
    Seu código deve retornar um vetor de previsões para toda linha de X.
    '''
    P = nd.exp(nd.dot(X, Theta.T))
    P = nd.clip(P, 1e-8, 1-1e-8)
    return (P.T / P.sum(axis=1)).T

In [10]:
# testes, não apague!

# Se X tem tamanho (n, f) a matriz theta é (c, f). c é o numéro de clases.
# X.T é (f, c). Assim X @ Theta.T -> (n, c)
# O resultado do softmax é a probabilidade de cada classe

Theta = nd.random.normal(shape=(4, X.shape[1]))
P = softmax(nd.array(X), Theta)
assert_equal((200, 4), P.shape)
assert_almost_equal(np.ones(len(P)), P.sum(axis=1).asnumpy(), 4) # verifica se toda linha soma == 1.

Agora implemente uma função de perda de entropia cruzada. Um truque para implementar a mesma é pensar nas respostas como uma matrix $\mathbf{Y}$. O tamanho de $\mathbf{Y}$ é `(n, c)`, ou seja, exemplos por classes. Cada vetor das linhas da matriz, simplesmente $\mathbf{y}_i$, é da forma one-hot `0, 0, 0, 1, 0`. Neste formato, apenas a classe do exemplo está setada como 1, todo o resto é zero. No exemplo, o elemento $i$ é da classe `3` (assumindo que a primeira classe é `0`). Vamos converter nossa resposta y em tal matriz.

In [11]:
import sklearn.preprocessing
hot = sklearn.preprocessing.OneHotEncoder(sparse=False)
# y[:, None] adiciona uma dimensão, vira uma matriz (n, 1). Cada linha é uma classe, e.g.: [[1], [0], [0], ...]
# O fit_transform vai converter em uma matriz (n, c).
Y = hot.fit_transform(y[:, None]) 
# fazendo Y ser mxnet
Y = nd.array(Y)
Y


[[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.

In [12]:
print(Y.shape)

(200, 2)


In [13]:
# fazendo X ser mxnet
X = nd.array(X)

In [14]:
print(X.shape)

(200, 20)


Agora podemos definir a entropia cruzada do caso multiclasse. Sendo:

$$\hat{\mathbf{p}}_i = < p(y_0|\mathbf{x}_i, \mathbf{\theta}_y)), \quad  p(y_1|\mathbf{x}_i, \mathbf{\theta}_y)), \quad \cdots, \quad p(y_{n-1}|\mathbf{x}_i, \mathbf{\theta}_y))>^t$$

Um vetor coluna com a probabilidade de cada classe para o exemplo $i$. Ou seja, o vetor tem `c` linhas. $\mathbf{y}^t_i$ é um vetor linha one-hot: e.g., `0, 0, 0, 1, 0`. O produto interno dos dois retorna a probabilidade da classe correta para cada exemplo: $\mathbf{y}^t_i \hat{\mathbf{p}}_i$. A média do log de cada probabilidade é a entropia cruzada!

$$CE(\theta \mid y_i, \mathbf{x}_i) = n^{-1} \sum_i \mathbf{y}^t_i \log(\, \hat{\mathbf{p}}_i\, )$$

Usando sua matriz `Y` acima. Basta fazer `Y * P` (multiplicação por elemento) para zerar qualquer probabilidade da classe errada. Depois disso, um `.sum(axis=1)` faz o produto interno para todos os exemplos.

Implemente a função `cross_entropy`.

In [15]:
def loss(X, Theta, Y):
    P = softmax(X, Theta)
    p = (Y * P).sum(axis=1)
    return -nd.log(p).mean()

In [16]:
# Testes
# O valor tem que ser um único número positivo
for _ in range(100):
    Theta = nd.random.normal(shape=(2, X.shape[1]))
    assert(loss(X, Theta, Y) >= 0)

In [17]:
def derivadas(X, Theta, Y):
    Theta.attach_grad()
    with mx.autograd.record():
        l = loss(X, Theta, Y)
    l.backward()
    return Theta.grad

In [18]:
loss(X, Theta, Y)


[ 6.40381336]
<NDArray 1 @cpu(0)>

In [19]:
derivadas(X, Theta, Y)


[[-1.23151088  0.45219558  0.15301538 -1.61669433  1.13179815  1.28182578
  -0.0508186  -0.2946412  -0.30722007 -1.62411141  0.42496043 -2.05265784
   0.39501527 -1.66971064  1.7529763  -1.8522203  -0.813833    2.04631901
  -0.66479421  1.16478693]
 [ 3.59458804 -3.61098671 -0.35592383 -2.59600973  4.53887081  4.72596884
   0.57829803 -2.03559113  2.40517831 -4.00310755 -4.62613964  2.83417749
  -3.28792882  3.65323663  0.87022269  2.53266978 -0.45683575 -3.00762415
  -2.0758276   3.2242322 ]]
<NDArray 2x20 @cpu(0)>

In [20]:
# Use essa função antes de executar o GD
def add_intercept(X):
    Xn = nd.zeros(shape=(X.shape[0], X.shape[1] + 1))
    Xn[:, 0]  = 1
    Xn[:, 1:] = X
    return Xn

In [21]:
Xn = add_intercept(X)
Theta = gd(derivadas, loss, Xn, Y, lambda_=0.005)

Iter 0; Loss =  inf

[ 0.69314718]
<NDArray 1 @cpu(0)> inf
Iter 1; Loss =  
[ 0.69314718]
<NDArray 1 @cpu(0)>

[ 0.45955646]
<NDArray 1 @cpu(0)> 
[ 0.69314718]
<NDArray 1 @cpu(0)>
Iter 2; Loss =  
[ 0.45955646]
<NDArray 1 @cpu(0)>

[ 0.4298825]
<NDArray 1 @cpu(0)> 
[ 0.45955646]
<NDArray 1 @cpu(0)>
Iter 3; Loss =  
[ 0.4298825]
<NDArray 1 @cpu(0)>

[ 0.41581097]
<NDArray 1 @cpu(0)> 
[ 0.4298825]
<NDArray 1 @cpu(0)>
Iter 4; Loss =  
[ 0.41581097]
<NDArray 1 @cpu(0)>

[ 0.4070515]
<NDArray 1 @cpu(0)> 
[ 0.41581097]
<NDArray 1 @cpu(0)>
Iter 5; Loss =  
[ 0.4070515]
<NDArray 1 @cpu(0)>

[ 0.40071407]
<NDArray 1 @cpu(0)> 
[ 0.4070515]
<NDArray 1 @cpu(0)>
Iter 6; Loss =  
[ 0.40071407]
<NDArray 1 @cpu(0)>

[ 0.39562535]
<NDArray 1 @cpu(0)> 
[ 0.40071407]
<NDArray 1 @cpu(0)>
Iter 7; Loss =  
[ 0.39562535]
<NDArray 1 @cpu(0)>

[ 0.39152324]
<NDArray 1 @cpu(0)> 
[ 0.39562535]
<NDArray 1 @cpu(0)>
Iter 8; Loss =  
[ 0.39152324]
<NDArray 1 @cpu(0)>

[ 0.38728118]
<NDArray 1 @cpu(0)> 
[ 0.39152324]

In [22]:
def previsoes(X, Theta):
    P = softmax(X, Theta)
    return P.argmax(axis=1)

In [23]:
y_p = previsoes(Xn, Theta)

In [24]:
y_p


[ 0.  1.  0.  1.  1.  0.  1.  1.  0.  1.  0.  1.  1.  0.  1.  1.  0.  1.
  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  1.  0.  1.  1.  1.  1.  1.  1.
  1.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  1.  0.  1.
  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  1.  0.  0.  1.  1.  0.  0.  0.
  1.  1.  1.  0.  1.  1.  1.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.
  1.  0.  0.  0.  1.  0.  1.  1.  1.  1.  0.  1.  0.  1.  1.  0.  1.  0.
  0.  0.  1.  0.  1.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.
  1.  0.  0.  1.  0.  0.  1.  0.  1.  1.  0.  0.  1.  0.  1.  1.  0.  1.
  1.  0.  1.  1.  1.  1.  0.  1.  0.  1.  0.  0.  1.  1.  0.  0.  0.  1.
  1.  0.  1.  1.  0.  1.  0.  1.  0.  0.  0.  1.  0.  0.  1.  0.  0.  1.
  1.  1.  0.  0.  0.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.  0.  1.
  0.  1.]
<NDArray 200 @cpu(0)>

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y, y_p.asnumpy()))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       100
          1       1.00      1.00      1.00       100

avg / total       1.00      1.00      1.00       200



### Com Gluon

Acima você deve ter visto uma classificação perfeita. Agora vamos testar em outro tipo de base. Primeiro vamos fazer o import do gluon. Gluon é uma API mais alto nível para criação de redes neurais. Vamos criar nosso softmax em gluon e avaliar em um data de dados de imagens com 10 classes. Você não precisa implementar nada. A ideia até mais em baixo é entender como a vida é mais simples ao usar a API do Gluon.

In [26]:
from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet.gluon import nn
from mxnet.gluon import utils

from mxnet import init
from mxnet import gluon

A criação de uma rede com gluon é trivial. Bastante similar com outras APIs de alto nível estilo Keras. Abaixo, defimos uma rede neural sequencial com uma única ativação densa de tamanho 10 saídas. Isto define uma função softmax de 10 classes.

In [27]:
net = nn.Sequential()
net.add(nn.Dense(10))
net.initialize(init.Normal(sigma=0.01)) # valores iniciais são uma normal

Agora indicamos que vamos fazer uso de uma entropia cruzada como função de perda.

In [28]:
cross_entropy = gloss.SoftmaxCrossEntropyLoss()

E que vamos treinar com gradiente descendente. SGD em particular!

In [29]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

Todos os passos que fizemos na mão se reduzem a poucas linhas

#### Carregando Dados

Agora vamos carregar os dados do Fashion Mnist. Leia [esta seção](http://d2l.ai/chapter_linear-networks/fashion-mnist.html) para uma descrição da base. O MXnet consegue baixar ela da internet.

Antes de usar a base, vamos definir uma função de transformação para converter a mesma de imagens (28, 28) para vetores de (28 * 28) posições. Além do mais, converter as unidades para float32. Bibliotecas de aprendizado profundo como mxnet são bem chatas com os tipos.

In [30]:
def transform(data, label):
    return data.astype('float32').reshape(28*28)/255, label.astype('float32')

Carregando a Base.

In [31]:
mnist_train = gdata.vision.FashionMNIST(train=True,
                                             transform=transform)
mnist_test = gdata.vision.FashionMNIST(train=False,
                                            transform=transform)

Com a classe `gluon.data.DataLoader` conseguimos iterar pela base em minibatches. Estamos usando aqui batches de tamanho 50.

In [32]:
minibatch_train = gluon.data.DataLoader(mnist_train, 50)
minibatch_test = gluon.data.DataLoader(mnist_test, 50)

#### Treinando

Podemos treinar com um laço. Abaixo detalhamos o mesmo.

In [33]:
iteracoes_treino = 5
for i in range(iteracoes_treino):         # número de iterações, não verificamos convergencia
    cumulative_loss = 0
    cumulative_size = 0
    for data, y in minibatch_train:       # para cada minibatch
        with mx.autograd.record():        # indicando que vamos derivar
            P = net(data)                 # execute o softmax, retorne as probabilidades
            loss = cross_entropy(P, y)    # compute a perda
        loss.backward()                   
        trainer.step(data.shape[0])       # atualize os parâmetros com a derivada
        cumulative_loss += nd.sum(loss).asscalar()
        cumulative_size += len(data)
    print('Iteração {}. Perda {}'.format(i, cumulative_loss / cumulative_size))

Iteração 0. Perda 735.7340666389465
Iteração 1. Perda 590.0368987083435
Iteração 2. Perda 560.9309377288819
Iteração 3. Perda 545.1596382808685
Iteração 4. Perda 534.6925950050354


Abaixo avaliamos a acurácia no teste.

Note o use da classe `mx.metric.Accuracy()`. A mesma acumula resultados para cada minibatch do teste. Sim, é esquisito, mas deixa a ideia de iterar por batches consistente em treino/teste.

In [34]:
acc = mx.metric.Accuracy()
for data, y in minibatch_test:
    P = net(data)
    pred = P.argmax(axis=1)
    acc.update(preds = pred, labels = y)
print(acc)

EvalMetric: {'accuracy': 0.84940000000000004}


## Perguntas

1. Altere o treino para computar a acurácia no mesmo
1. Brinque com a taxa de aprendizado do SGD e o número de iterações
1. Qual o impacto na acurácia do treino/teste?